In [ ]:
# Algorithmic Trading with Relative Strength Index in Python

In [ ]:
# Website Link: https://medium.com/codex/algorithmic-trading-with-relative-strength-index-in-python-d969cf22dd85

In [ ]:
# Step-1: Importing Packages

import pandas as pd 
import matplotlib.pyplot as plt
import requests
import numpy as np
from math import floor
from termcolor import colored as cl 

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20, 10)

# Step-2: Extracting Data from Alpha Vantage

def get_historical_data(symbol, start_date = None):
    api_key = open(r'api_key.txt')
    api_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full'
    raw_df = requests.get(api_url).json()
    df = pd.DataFrame(raw_df[f'Time Series (Daily)']).T
    df = df.rename(columns = {'1. open': 'open', '2. high': 'high', '3. low': 'low', '4. close': 'close', '5. adjusted close': 'adj close', '6. volume': 'volume'})
    for i in df.columns:
        df[i] = df[i].astype(float)
    df.index = pd.to_datetime(df.index)
    df = df.iloc[::-1].drop(['7. dividend amount', '8. split coefficient'], axis = 1)
    if start_date:
        df = df[df.index >= start_date]
    return df

ibm = get_historical_data('IBM', '2020-01-01')
print(ibm)

# Step-3: RSI Calculation
#
# Calculating the returns of the stock using the ‘diff’ function provided by the Pandas package and stored 
# it into the ‘ret’ variable. This function basically subtracts the current value from the previous value.
def get_rsi(close, lookback):
    ret = close.diff()
    up = []
    down = []
    
# Passing a for-loop on the ‘ret’ variable to distinguish gains from losses and append those values to the 
# concerning variable (‘up’ or ‘down’). 
    for i in range(len(ret)):
        if ret[i] < 0:
            up.append(0)
            down.append(ret[i])
        else:
            up.append(ret[i])
            down.append(0)
    up_series = pd.Series(up)
    down_series = pd.Series(down).abs()
    
# Calculating the Exponential Moving Averages for both the ‘up’ and ‘down’ using the ‘ewm’ function 
# provided by the Pandas package and stored them into the ‘up_ewm’ and ‘down_ewm’ variable respectively.
# Using these calculated EMAs, we are determining the Relative Strength by following the formula we 
# discussed before and stored it into the ‘rs’ variable.
    up_ewm = up_series.ewm(com = lookback - 1, adjust = False).mean()
    down_ewm = down_series.ewm(com = lookback - 1, adjust = False).mean()
    rs = up_ewm/down_ewm
    
# Calculating the RSI values by following its formula.
# Returning the calculated Relative Strength Index values in the form of a Pandas dataframe. 
    rsi = 100 - (100 / (1 + rs))
    rsi_df = pd.DataFrame(rsi).rename(columns = {0:'rsi'}).set_index(close.index)
    rsi_df = rsi_df.dropna()
    return rsi_df[3:]

# Calling the created function to store the RSI values of IBM with 14 as the lookback period.
ibm['rsi_14'] = get_rsi(ibm['close'], 14)
ibm = ibm.dropna()
print(ibm)


# Step-4: RSI Plot
#
# Plot the calculated Relative Strength Index values of IBM 
# Chart is separated into two panels: The above panel with the closing price of IBM and the 
# lower panel with the calculated RSI 14 values of IBM.
ax1 = plt.subplot2grid((10,1), (0,0), rowspan = 4, colspan = 1)
ax2 = plt.subplot2grid((10,1), (5,0), rowspan = 4, colspan = 1)
ax1.plot(ibm['close'], linewidth = 2.5)
ax1.set_title('IBM CLOSE PRICE')
ax2.plot(ibm['rsi_14'], color = 'orange', linewidth = 2.5)
ax2.axhline(30, linestyle = '--', linewidth = 1.5, color = 'grey')
ax2.axhline(70, linestyle = '--', linewidth = 1.5, color = 'grey')
ax2.set_title('IBM RELATIVE STRENGTH INDEX')
plt.show()


# Step-5: Creating the trading strategy 
#
# Defining a function named ‘implement_rsi_strategy’ which takes the stock prices (‘prices’), 
# and the RSI values (‘rsi’) as parameters.
# Creating three empty lists (buy_price, sell_price, and rsi_signal) in which the values will 
# be appended while creating the trading strategy.
def implement_rsi_strategy(prices, rsi):    
    buy_price = []
    sell_price = []
    rsi_signal = []
    signal = 0

    for i in range(len(rsi)):
        if rsi[i-1] > 30 and rsi[i] < 30:
            
# If the condition to buy the stock gets satisfied, the buying price will be appended to the 
# ‘buy_price’ list, and the signal value will be appended as 1 representing to buy the stock.
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                rsi_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                rsi_signal.append(0)
                
# If the condition to sell the stock gets satisfied, the selling price will be appended to 
# the ‘sell_price’ list, and the signal value will be appended as -1 representing to sell the stock.                
        elif rsi[i-1] < 70 and rsi[i] > 70:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                rsi_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                rsi_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            rsi_signal.append(0)

# Returning the lists appended with values. 
# Calling the created function and stored the values into their respective variables.
    return buy_price, sell_price, rsi_signal
            

buy_price, sell_price, rsi_signal = implement_rsi_strategy(ibm['close'], ibm['rsi_14'])


# Step-6: Plotting the trading signals

ax1 = plt.subplot2grid((10,1), (0,0), rowspan = 4, colspan = 1)
ax2 = plt.subplot2grid((10,1), (5,0), rowspan = 4, colspan = 1)
ax1.plot(ibm['close'], linewidth = 2.5, color = 'skyblue', label = 'IBM')
ax1.plot(ibm.index, buy_price, marker = '^', markersize = 10, color = 'green', label = 'BUY SIGNAL')
ax1.plot(ibm.index, sell_price, marker = 'v', markersize = 10, color = 'r', label = 'SELL SIGNAL')
ax1.set_title('IBM RSI TRADE SIGNALS')
ax2.plot(ibm['rsi_14'], color = 'orange', linewidth = 2.5)
ax2.axhline(30, linestyle = '--', linewidth = 1.5, color = 'grey')
ax2.axhline(70, linestyle = '--', linewidth = 1.5, color = 'grey')
plt.show()


# Step-7: Creating our Position

# Creating an empty list named ‘position’.
# Passing two for-loops, one is to generate values for the ‘position’ list to just match the length of the ‘signal’ list.
# The other for-loop is the one we are using to generate actual position values.
# Inside the second for-loop, we are iterating over the values of the ‘signal’ list, and the values of the ‘position’ 
# list get appended concerning which condition gets satisfied. The value of the position remains 1 if we hold the stock 
# or remains 0 if we sold or don’t own the stock.
position = []
for i in range(len(rsi_signal)):
    if rsi_signal[i] > 1:
        position.append(0)
    else:
        position.append(1)
        
for i in range(len(ibm['close'])):
    if rsi_signal[i] == 1:
        position[i] = 1
    elif rsi_signal[i] == -1:
        position[i] = 0
    else:
        position[i] = position[i-1]

# Finally, we are doing some data manipulations to combine all the created lists into one dataframe.
rsi = ibm['rsi_14']
close_price = ibm['close']
rsi_signal = pd.DataFrame(rsi_signal).rename(columns = {0:'rsi_signal'}).set_index(ibm.index)
position = pd.DataFrame(position).rename(columns = {0:'rsi_position'}).set_index(ibm.index)

frames = [close_price, rsi, rsi_signal, position]
strategy = pd.concat(frames, join = 'inner', axis = 1)

print(strategy.head())



# Step-8: Backtesting
#
# Calculating the returns of the IBM stock using the ‘diff’ function provided by the NumPy package 
# and we have stored it as a dataframe into the ‘ibm_ret’ variable.
ibm_ret = pd.DataFrame(np.diff(ibm['close'])).rename(columns = {0:'returns'})
rsi_strategy_ret = []

# We are passing a for-loop to iterate over the values of the ‘ibm_ret’ variable to calculate the returns we gained
# from our Relative Strength Index trading strategy, and these returns values are appended to the ‘rsi_strategy_ret’ list.
for i in range(len(ibm_ret)):
    returns = ibm_ret['returns'][i]*strategy['rsi_position'][i]
    rsi_strategy_ret.append(returns)

# Converting the ‘rsi_strategy_ret’ list into a dataframe and stored it into the ‘rsi_strategy_ret_df’ variable. 
rsi_strategy_ret_df = pd.DataFrame(rsi_strategy_ret).rename(columns = {0:'rsi_returns'})

# We are going to backtest our strategy by investing a hundred thousand USD into our trading strategy. 
# So first, we are storing the amount of investment into the ‘investment_value’ variable. After that, 
# we are calculating the number of IBM stocks we can buy using the investment amount. 
# Using the ‘floor’ function, we can cut out the decimals. 
investment_value = 100000
number_of_stocks = floor(investment_value/ibm['close'][-1])
rsi_investment_ret = []

# We are passing a for-loop to find the investment returns followed by some data manipulations tasks.
for i in range(len(rsi_strategy_ret_df['rsi_returns'])):
    returns = number_of_stocks*rsi_strategy_ret_df['rsi_returns'][i]
    rsi_investment_ret.append(returns)

rsi_investment_ret_df = pd.DataFrame(rsi_investment_ret).rename(columns = {0:'investment_returns'})
total_investment_ret = round(sum(rsi_investment_ret_df['investment_returns']), 2)
profit_percentage = floor((total_investment_ret/investment_value)*100)

# Printing the total return we got by investing a hundred thousand into our trading strategy and it is revealed that we have made 
# an approximate profit of twenty-one thousand USD with a profit percentage of twenty-one percent in one year.
print(cl('Profit gained from the RSI strategy by investing $100k in IBM : {}'.format(total_investment_ret), attrs = ['bold']))
print(cl('Profit percentage of the RSI strategy : {}%'.format(profit_percentage), attrs = ['bold']))